In [2]:
import pandas as pd
import sqlite3

# Load the corrected Excel file (with full refunds adjusted)
excel_path = r'C:\Users\rishitha.p\OneDrive - TECHNOVERT SOLUTIONS PRIVATE LIMITED\MULTIAGENT\ECOM\Ecommerce_Customer_Care_Full_Refund_Final.xlsx'
xls = pd.ExcelFile(excel_path)

# Connect or create SQLite DB
conn = sqlite3.connect('ecommerce_final.db')
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON")



In [3]:
# Step 1: Create tables with PK and FK
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    phone_number TEXT,
    address TEXT,
    created_at TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    description TEXT,
    price REAL,
    stock_quantity INTEGER,
    category TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    product_id INTEGER,
    order_status TEXT,
    total_amount REAL,
    order_date TEXT,
    delivery_date TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS refunds (
    refund_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    refund_status TEXT,
    reason TEXT,
    refund_amount REAL,
    requested_at TEXT,
    processed_at TEXT,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

conn.commit()



In [4]:
# Step 2: Insert data from each sheet into corresponding tables
for sheet in ['Customers', 'Products', 'Orders', 'Refunds']:
    df = pd.read_excel(xls, sheet_name=sheet)
    df.columns = df.columns.str.strip()  # Clean column names
    df.to_sql(sheet.lower(), conn, if_exists='append', index=False)
    print(f"✅ Inserted {sheet} data successfully")

conn.commit()
conn.close()
print("🎉 Database 'ecommerce_final.db' created with Primary & Foreign Keys!")


✅ Inserted Customers data successfully
✅ Inserted Products data successfully
✅ Inserted Orders data successfully
✅ Inserted Refunds data successfully
🎉 Database 'ecommerce_final.db' created with Primary & Foreign Keys!


In [5]:
conn = sqlite3.connect('ecommerce_final.db')
print(conn.execute("PRAGMA foreign_key_list('refunds')").fetchall())
conn.close()


[(0, 0, 'products', 'product_id', 'product_id', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'orders', 'order_id', 'order_id', 'NO ACTION', 'NO ACTION', 'NONE')]
